# Day 13 - Weather API

In [19]:
import openmeteo_requests
from openmeteo_sdk.Variable import Variable
import pandas as pd
import altair as alt

om = openmeteo_requests.Client()
params = {
    "latitude": 55.70,
    "longitude": 12.53,
    "hourly": ["temperature_2m", "precipitation", "wind_speed_10m"],
    "current": ["temperature_2m", "relative_humidity_2m"]
}

responses = om.weather_api("https://api.open-meteo.com/v1/forecast", params=params)
response = responses[0]

In [36]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy() # unit: celcius
hourly_precipitation = hourly.Variables(1).ValuesAsNumpy() # unit: mm
hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy() # unit: km/h

In [37]:

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["precipitation"] = hourly_precipitation

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe

,date,temperature_2m,wind_speed_10m,precipitation
0,2025-06-24 00:00:00+00:00,13.443500,21.959999,0.0
1,2025-06-24 01:00:00+00:00,12.993500,22.680000,0.0
2,2025-06-24 02:00:00+00:00,13.343500,20.880001,0.0
3,2025-06-24 03:00:00+00:00,13.593500,18.719999,0.0
4,2025-06-24 04:00:00+00:00,13.943500,18.359999,0.0
...,...,...,...,...
163,2025-06-30 19:00:00+00:00,19.850000,6.034700,0.0
164,2025-06-30 20:00:00+00:00,18.900000,5.483356,0.0
165,2025-06-30 21:00:00+00:00,17.950001,4.829907,0.0
166,2025-06-30 22:00:00+00:00,17.100000,3.893995,0.0


In [54]:
# make the chart
chart1 = alt.Chart(hourly_dataframe).mark_line().encode(
    x='date',
    y='temperature_2m',
)
chart2 = alt.Chart(hourly_dataframe).mark_line().encode(
    x='date',
    y='wind_speed_10m'
)

chart3 = alt.Chart(hourly_dataframe).mark_bar().encode(
    x='date',
    y='precipitation'
)

alt.hconcat(chart1, chart2, chart3)

alt.HConcatChart(...)